# Test notebook

__A. J. Tropiano [tropiano.4@osu.edu]__<br/>
__March 17, 2022__

The purpose of this notebook is for testing out code and/or scripts.

_Last update: April 20, 2022_

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

In [2]:
# Imports from A.T. codes
import densities
from dmd import Deuteron
from momentum_distributions import MomentumDistribution
from potentials import Potential
# You could import modules but I want a list of scripts here
# Check these off the list as you go through Jupyter notebooks generating
# figures
import modules.a2
import modules.figures
import modules.fourier_transform
from modules.integration import gaussian_quadrature_mesh
import modules.labels
import modules.long_distance_operators
from magnus import Magnus, MagnusSplit
import modules.momentum_projection_operator
import modules.phase_shifts
import modules.sdt
from pmd import Pair
from snmd import SingleNucleon
from srg import SRG, get_transformation
import modules.tools
import modules.wave_function
# add missing ones

For now let's illustrate the overhaul by testing scripts and showing how to use them.

### Testing momentum distributions code

In [3]:
# # Test default interaction and SRG evolution

# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
# channels = ('1S0', '3S1')
# generator = 'Wegner'
# lamb = 1.35

In [4]:
# # Get momentum values (channel argument doesn't matter here)
# potential = Potential(kvnn, '1S0', kmax, kmid, ntot)
# q_array, _ = potential.load_mesh()

# # Set C.o.M. momentum too
# Q_max = 2.0
# ntot_Q = 40
# Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

In [5]:
# # Do Ca48 with SLy4 densities

# nucleus_name = 'Ca48'
# Z, N = 20, 28
# density = 'SLY4'

In [6]:
# # Generate new files and compare to old ones
# # Single-nucleon matches old data exactly

# snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)

# for nucleon in ('proton', 'neutron'):
    
#     t0 = time.time()
    
#     n_array = snmd.compute_momentum_distribution(
#         q_array, nucleon, nucleus_name, Z, N, density, save=True)
    
#     t1 = time.time()
#     mins = (t1-t0)/60
#     print(f'Done with {nucleon} after {mins:.2f} minutes.')

In [7]:
# # Pair matches old data exactly

# pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)

# for pair in ('pn', 'pp', 'nn'):
    
#     t0 = time.time()
            
#     # Q > 0
#     n_array = pmd.compute_momentum_distribution(
#         q_array, Q_array, pair, nucleus_name, Z, N, density, save=True)
    
#     # Q = 0
#     n_Q0_array = pmd.compute_momentum_distribution_Q0(
#         q_array, pair, nucleus_name, Z, N, density, save=True)

#     t1 = time.time()
#     mins = (t1-t0)/60
#     print(f'Done with {pair} after {mins:.2f} minutes.')

In [8]:
# # Deuteron matches old data exactly

# dmd = Deuteron(kvnn, kmax, kmid, ntot, generator, lamb)
    
# t0 = time.time()
        
# n_array = dmd.compute_momentum_distribution(q_array, save=True)
            
# t1 = time.time()
# mins = (t1-t0)/60
        
# print(f'Done after {mins:.2f} minutes.')

### Generate momentum distribution data

In [9]:
# kvnns = (2, 3, 4, 5, 6, 7, 79, 110, 111, 112, 113, 222, 224)
# Note: Each kvnn + Gogny run takes about 2.7 hours
kvnns = (110, 111, 112, 113, 222, 224)
kmax, kmid, ntot = 15.0, 3.0, 120
channels = ('1S0', '3S1')
generator = 'Wegner'
lamb = 1.35
density = 'Gogny'
nuclei_gogny = (
    ('He4', 2, 2), ('Li7', 3, 4), ('Be9', 4, 5), ('C12', 6, 6), ('O16', 8, 8),
    ('Al27', 13, 14), ('Ca40', 20, 20), ('Ca48', 20, 28), ('Ti48', 22, 26),
    ('Fe56', 26, 30), ('Cu63', 29, 34), ('Ag107', 47, 60), ('Sn118', 50, 68),
    ('Ce140', 58, 82), ('Ta181', 73, 108), ('Au197', 79, 118),
    ('Pb208', 82, 126), ('U238', 92, 146)
)

potential = Potential(6, '1S0', kmax, kmid, ntot)
q_array, _ = potential.load_mesh()  # This will be the same in every case

# Set C.o.M. momentum too
Q_max = 2.0
ntot_Q = 40
Q_array, _ = gaussian_quadrature_mesh(Q_max, ntot_Q)

# Generate files
for kvnn in kvnns:
    
    print(f'Starting kvnn = {kvnn}.')
    t0_kvnn = time.time()

    dmd = Deuteron(kvnn, kmax, kmid, ntot, generator, lamb)
    snmd = SingleNucleon(kvnn, kmax, kmid, ntot, channels, generator, lamb)
    pmd = Pair(kvnn, kmax, kmid, ntot, channels, generator, lamb)

    # Deuteron first
    t0_d = time.time()
    n_array = dmd.compute_momentum_distribution(q_array, save=True)   
    t1_d = time.time()
    mins = (t1_d-t0_d)/60
    print(f'Done with deuteron after {mins:.2f} minutes.')
    
    for nucleus in nuclei_gogny:
        
        nucleus_name = nucleus[0]
        Z = nucleus[1]
        N = nucleus[2]
        
        t0_nucleus = time.time()
        
        # Single-nucleon
        for nucleon in ('proton', 'neutron'):
            
            n_array = snmd.compute_momentum_distribution(
                q_array, nucleon, nucleus_name, Z, N, density, save=True)
        
        # Pair
        for pair in ('pn', 'pp', 'nn'):
        
            # Q > 0
            n_array = pmd.compute_momentum_distribution(
                q_array, Q_array, pair, nucleus_name, Z, N, density,
                save=True)
            # Q = 0
            n_Q0_array = pmd.compute_momentum_distribution_Q0(
                    q_array, pair, nucleus_name, Z, N, density, save=True)

        t1_nucleus = time.time()
        mins = (t1_nucleus-t0_nucleus)/60
        print(f'Done with {nucleus_name} after {mins:.2f} minutes.')
        
    t1_kvnn = time.time()
    mins = (t1_kvnn-t0_kvnn)/60
    print(f'Done with kvnn = {kvnn} after {mins:.2f} minutes.\n')

Starting kvnn = 6.
Done with deuteron after 0.86 minutes.
Done with He4 after 8.44 minutes.
Done with Li7 after 8.36 minutes.
Done with Be9 after 8.32 minutes.
Done with C12 after 8.50 minutes.
Done with O16 after 8.37 minutes.
Done with Al27 after 8.36 minutes.
Done with Ca40 after 8.54 minutes.
Done with Ca48 after 8.48 minutes.
Done with Ti48 after 8.55 minutes.
Done with Fe56 after 8.52 minutes.
Done with Cu63 after 8.59 minutes.
Done with Ag107 after 8.45 minutes.
Done with Sn118 after 8.42 minutes.
Done with Ce140 after 8.56 minutes.
Done with Ta181 after 8.51 minutes.
Done with Au197 after 8.70 minutes.
Done with Pb208 after 8.61 minutes.
Done with U238 after 8.73 minutes.
Done with kvnn = 6 after 153.97 minutes.

Starting kvnn = 7.
Done with deuteron after 0.87 minutes.
Done with He4 after 8.57 minutes.
Done with Li7 after 8.50 minutes.
Done with Be9 after 8.46 minutes.
Done with C12 after 8.42 minutes.
Done with O16 after 8.39 minutes.
Done with Al27 after 8.39 minutes.
Done w

OSError: ../potentials/vsrg_macos/vsrg_kvnn_79_lam12.0_kmax15_kmid3_ntot120/vsrg_3S1_kvnn_79_lam12.0_reg_0_3_0_mesh.out not found.

In [ ]:
# Do additional two densities (SLy4 and VMC) for AV18

In [ ]:
# Do other iterations of channels with AV18 and SLY4

In [ ]:
# Do inverse-SRG with \delta \lambda = (6, 5.5, ..., 3.5, 3.0) for kvnn and
# kvnn_inv pairs of (111, 6), (111, 113), (113, 6), (222, 6)

In [ ]:
# Do block-diagonal and \lambda dependence for AV18 with Gogny density
# Don't repeat \lambda = 1.35 for band-diagonal